In [16]:
import warnings
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

warnings.filterwarnings("ignore", category=UserWarning)

In [17]:
import torch
import numpy as np
from transformers import BertTokenizerFast, BertForMultipleChoice, Trainer, TrainingArguments
from datasets import load_dataset


In [18]:
# Загружаем предварительно подготовленный датасет
ds = load_dataset("PARus")

model_name = "DeepPavlov/rubert-base-cased"
# Загружаем предобученную модель
model = BertForMultipleChoice.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    model = model.to("cuda")
# Подготавливаем токенизатор
tokenizer = BertTokenizerFast.from_pretrained(model_name)

max_length_choice1 = max([len(dict(i).get('choice1')) for i in ds['train']])
max_length_choice2 = max([len(dict(i).get('choice2')) for i in ds['train']])
max_length = max(max_length_choice1, max_length_choice2)
max_length

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


64

In [19]:
# Функция подготовки выборок для multiple choice task
def prepare_features(batch):
    # Конструируем prompt для каждого варианта выбора
    choices = [batch['choice1'], batch['choice2']]
    first_sentences = [batch['premise']] * len(choices)
    second_sentences = [f"{batch['question']} {choice}" for choice in choices]

    # Токенизируем с сохранением связи индексов
    inputs = tokenizer(first_sentences,
                       second_sentences,
                       truncation="only_second",
                       max_length=max_length,
                       padding="max_length"
                       )

    # Добавляем метку правильного ответа
    label = batch.get('label')  # Если label отсутствует, значит тестовая выборка
    if label is not None:
        inputs['labels'] = label
    return inputs

In [5]:
# Обрабатываем датасеты
for split_name in ['train', 'validation']:
    ds[split_name] = ds[split_name].map(
        prepare_features,
        batched=False,
        remove_columns=['premise', 'choice1', 'choice2', 'question', 'label']
    )

ds["validation"] = ds["validation"].shuffle()
ds["train"] = ds["train"].shuffle()

In [6]:
# Преобразуем объект в PyTorch TensorDataset
ds["train"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])
ds["validation"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])

ds["train"]

Dataset({
    features: ['idx', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 400
})

In [7]:
# Определение метрик точности (accuracy)
from sklearn.metrics import accuracy_score


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

In [8]:
# Аргументы обучения
training_args = TrainingArguments(
    output_dir='./rubert_finetuned_PARus',
    eval_strategy="epoch",
    save_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    learning_rate=1e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    greater_is_better=True,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Создаем тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],
    compute_metrics=compute_metrics,
)

In [9]:
# Начинаем обучение
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.677063,0.600000
2,No log,0.663547,0.620000
3,No log,0.660256,0.610000
4,No log,0.663437,0.670000
5,No log,0.709936,0.630000
6,No log,0.705730,0.680000
7,No log,0.741314,0.690000
8,No log,0.756240,0.680000
9,No log,0.749731,0.670000
10,No log,0.744783,0.680000


TrainOutput(global_step=250, training_loss=0.4162550048828125, metrics={'train_runtime': 3264.8735, 'train_samples_per_second': 1.225, 'train_steps_per_second': 0.077, 'total_flos': 263108692992000.0, 'train_loss': 0.4162550048828125, 'epoch': 10.0})

In [10]:
# Оцениваем модель на валидирующем наборе
eval_results = trainer.evaluate()
print(f'Точность в наборе для проверки: {eval_results["eval_accuracy"]:.3f}')


Точность в наборе для проверки: 0.690


In [11]:
# Сохранение финальной модели
trainer.save_model("./rubert_finetuned_PARus/final_model")

In [20]:
# Загружаем предобученную модель
save_model_name = "./rubert_finetuned_PARus/final_model"
# Загрузить модель из сохранённого каталога:
loaded_model = BertForMultipleChoice.from_pretrained(save_model_name)
# Также загрузить токенизатор из сохранённого каталога:
loaded_tokenizer = BertTokenizerFast.from_pretrained(model_name)

In [21]:
# Обрабатываем датасеты
ds["test"] = ds["test"].map(
        prepare_features,
        batched=False,
        remove_columns=['premise', 'choice1', 'choice2', 'question', 'label']
    )

ds["test"]

Dataset({
    features: ['idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 500
})

In [22]:
loaded_model.eval()

training_args = TrainingArguments(
    output_dir='./rubert_finetuned_PARus',
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 4,
    dataloader_drop_last = False,
)

# Создаем тренера
trainer = Trainer(
    model = loaded_model,
    args = training_args,
)

In [23]:
test_results_content = trainer.predict(ds["test"])[0]
test_results_content

array([[-1.35785043e+00, -1.39057636e+00],
       [-1.40500677e+00, -1.40225220e+00],
       [ 1.38050222e+00,  1.37695622e+00],
       [ 1.34420156e+00, -3.56237769e-01],
       [-9.26094711e-01, -4.45522726e-01],
       [-1.39719927e+00, -1.39763892e+00],
       [ 1.31414461e+00,  1.27292657e+00],
       [-1.39193821e+00, -6.48284680e-04],
       [ 2.73627162e-01, -1.26366329e+00],
       [-1.26657677e+00,  9.37433660e-01],
       [ 6.30275667e-01, -1.40294600e+00],
       [-1.40785933e+00,  1.40006495e+00],
       [ 9.94262874e-01, -3.67544770e-01],
       [ 7.67251909e-01,  9.17282045e-01],
       [ 1.37353873e+00,  1.39187527e+00],
       [ 1.38135624e+00,  8.56581509e-01],
       [ 8.96063805e-01, -2.59354413e-01],
       [ 1.14000750e+00,  1.10245705e+00],
       [ 1.16258931e+00, -1.39287663e+00],
       [ 7.07441092e-01,  2.10348684e-02],
       [ 1.39207339e+00, -1.16884470e+00],
       [ 1.31097698e+00, -1.40029442e+00],
       [-8.18131506e-01, -1.30375636e+00],
       [-1.

In [24]:
# Написание функции для получения предикта
def get_prediction():
    test_pred = trainer.predict(ds["test"])
    labels = np.argmax(test_pred.predictions, axis=-1)
    return labels


pred = get_prediction()

In [25]:
result_test = ds["test"]
result_test = result_test.add_column("label", pred)
result_test = result_test.remove_columns(['input_ids', 'token_type_ids', 'attention_mask'])
result_test

Dataset({
    features: ['idx', 'label'],
    num_rows: 500
})

In [26]:
result_test.to_json('PARus.jsonl')

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

10890